<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250625_Dataflow_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 指定所用模組版本

In [1]:
!pip install protobuf --quiet
!pip install numpy>=1.26.0 --quiet
!pip install apache_beam==2.60.0 --quiet
!pip install tensorflow==2.12.0 --quiet
# To use the newly installed versions, restart the runtime.
exit()

### 授權帳號權限

In [1]:
from google.colab import auth
auth.authenticate_user()

### 導入模組與常數

In [2]:
import argparse
from typing import Dict, Text, Any, Tuple, List
import numpy

from google.protobuf import text_format

import tensorflow as tf
from tensorflow import keras
import apache_beam as beam
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerNumpy
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerTensor
from apache_beam.options.pipeline_options import PipelineOptions

project = "tibame-gad251-31-0625"
bucket = "tibame-gad251-31-storage"

save_model_dir_multiply = f'gs://{bucket}/tf-inference/model/multiply_five/v1/'
save_weights_dir_multiply = f'gs://{bucket}/tf-inference/weights/multiply_five/v1/'


#### 建立Tensorflow模型

In [3]:
# Create training data that represents the 5 times multiplication table for the numbers 0 to 99.
# x is the data and y is the labels.
x = numpy.arange(0, 100)   # Examples
y = x * 5          # Labels

# Use create_model to build a simple linear regression model.
# Note that the model has a shape of (1) for its input layer and expects a single int64 value.
def create_model():
  input_layer = keras.layers.Input(shape=(1), dtype=tf.float32, name='x')
  output_layer= keras.layers.Dense(1)(input_layer)
  model = keras.Model(input_layer, output_layer)
  model.compile(optimizer=tf.optimizers.Adam(), loss='mean_absolute_error')
  return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x (InputLayer)              [(None, 1)]               0         
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


#### 測試模型狀況

In [4]:
model.fit(x, y, epochs=500, verbose=0)
test_examples =[20, 40, 60, 90]
value_to_predict = numpy.array(test_examples, dtype=numpy.float32)
predictions = model.predict(value_to_predict)

print('Test Examples ' + str(test_examples))
print('Predictions ' + str(predictions))

1/1 [==============================] - 0s 86ms/step
Test Examples [20, 40, 60, 90]
Predictions [[ 26.397907]
 [ 50.799816]
 [ 75.20173 ]
 [111.80459 ]]


#### 儲存模型

In [5]:
model.save(save_model_dir_multiply)
#model_handler = TFModelHandlerNumpy(path_to_weights, model_type=ModelType.SAVED_WEIGHTS, create_model_fn=build_tensorflow_model)
model.save_weights(save_weights_dir_multiply)

### 利用管道模型測試

In [6]:
class FormatOutput(beam.DoFn):
  def process(self, element, *args, **kwargs):
     """格式化推論結果輸出。"""
     yield "範例值為 {example}，預測結果為 {prediction}".format(example=element.example, prediction=element.inference)

# 建立輸入範例數據
examples = numpy.array([20, 40, 60, 90], dtype=numpy.float32)

# 設定 TensorFlow 模型處理器
model_handler = TFModelHandlerNumpy(save_model_dir_multiply)

with beam.Pipeline() as p:
    _ = (p
         | "建立輸入數據" >> beam.Create(examples)
         | "執行模型推論" >> RunInference(model_handler)
         | "格式化輸出結果" >> beam.ParDo(FormatOutput())
         | "輸出結果" >> beam.Map(print)
        )

範例值為 20.0，預測結果為 [26.397907]
範例值為 40.0，預測結果為 [50.799816]
範例值為 60.0，預測結果為 [75.20173]
範例值為 90.0，預測結果為 [111.80459]


### 利用模型權重推論

In [7]:
from apache_beam.ml.inference.tensorflow_inference import ModelType

# 建立輸入範例數據
examples = numpy.array([20, 40, 60, 90], dtype=numpy.float32)

# 設定 TensorFlow 模型處理器，使用儲存的權重進行推論
model_handler = TFModelHandlerNumpy(
    save_weights_dir_multiply,
    model_type=ModelType.SAVED_WEIGHTS,
    create_model_fn=create_model
)

with beam.Pipeline() as p:
    _ = (p
         | "建立輸入數據" >> beam.Create(examples)
         | "執行模型推論" >> RunInference(model_handler)
         | "格式化輸出結果" >> beam.ParDo(FormatOutput())
         | "輸出結果" >> beam.Map(print)
        )

範例值為 20.0，預測結果為 [26.397907]
範例值為 40.0，預測結果為 [50.799816]
範例值為 60.0，預測結果為 [75.20173]
範例值為 90.0，預測結果為 [111.80459]


### 利用含KEY資料測試模型

In [8]:
class FormatOutputKeyed(FormatOutput):
  # 繼承自 FormatOutput。
  def process(self, tuple_in: Tuple):
    key, element = tuple_in
    output = super().process(element)
    yield "{} : {}".format(key, [op for op in output])

# 建立輸入範例數據（包含鍵值）
examples = numpy.array([(1, 20), (2, 40), (3, 60), (4, 90)], dtype=numpy.float32)

# 設定鍵值模型處理器
keyed_model_handler = KeyedModelHandler(TFModelHandlerNumpy(save_model_dir_multiply))

with beam.Pipeline() as p:
    _ = (p
         | "建立輸入數據" >> beam.Create(examples)
         | "執行模型推論" >> RunInference(keyed_model_handler)
         | "格式化輸出結果（含鍵值）" >> beam.ParDo(FormatOutputKeyed())
         | "輸出結果" >> beam.Map(print)
        )


1.0 : ['範例值為 20.0，預測結果為 [26.397907]']
2.0 : ['範例值為 40.0，預測結果為 [50.799816]']
3.0 : ['範例值為 60.0，預測結果為 [75.20173]']
4.0 : ['範例值為 90.0，預測結果為 [111.80459]']
